# Clustering Crypto

In [22]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [23]:
# Load the crypto_data.csv dataset.
file = 'crypto_data.csv'
df = pd.read_csv(file)
df.set_index('Unnamed: 0', inplace=True)
df


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000


In [24]:
# Keep all the cryptocurrencies that are being traded.
df = df[df['IsTrading'] == True]


In [25]:
# Keep all the cryptocurrencies that have a working algorithm.
df['Algorithm'] = df['Algorithm'].dropna()
df

C:\Users\janze\AppData\Local\Temp\ipykernel_20376\3162967657.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Algorithm'] = df['Algorithm'].dropna()


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,,
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610


In [26]:
# Remove the "IsTrading" column. 
df = df.drop(columns = 'IsTrading')

In [27]:
# Remove rows that have at least 1 null value.
df.dropna(inplace=True)
df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [28]:
# Keep the rows where coins are mined.
df = df[df['TotalCoinsMined'] > 0]
df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [29]:
# Create a new DataFrame that holds only the cryptocurrencies names.
df_names = df['CoinName']
df_names

Unnamed: 0
42          42 Coin
404         404Coin
1337      EliteCoin
BTC         Bitcoin
ETH        Ethereum
           ...     
ZEPH         ZEPHYR
GAP         Gapcoin
BDX          Beldex
ZEN         Horizen
XBC     BitcoinPlus
Name: CoinName, Length: 532, dtype: object

In [30]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
df.drop(columns='CoinName', axis=1, inplace=True)
df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610


In [31]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(df, columns=['Algorithm', 'ProofType'], dtype=float)
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
404,1.055185e+09,532000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1337,2.927942e+10,314159265359,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BTC,1.792718e+07,21000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ETH,1.076842e+08,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GAP,1.493105e+07,250000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BDX,9.802226e+08,1400222610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# Standardize the data with StandardScaler().
scaler=StandardScaler()
scaled_df = scaler.fit_transform(X)

### Deliverable 2: Reducing Data Dimensions Using PCA

In [33]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)


In [34]:
# Create a DataFrame with the three principal components.
df_pca = pca.fit_transform(scaled_df)
df_pca = pd.DataFrame(df_pca, columns=["PC 1", "PC 2", "PC 3"], index=df_names)
df_pca.head()

,PC 1,PC 2,PC 3
CoinName,,,
42 Coin,-0.355362,1.027483,-0.542426
404Coin,-0.338728,1.027648,-0.542770
EliteCoin,2.343712,1.503374,-0.590403
Bitcoin,-0.131143,-1.333695,0.214659
Ethereum,-0.137197,-2.031783,0.401093


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [35]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1,11))

for i in k:
    km = KMeans(n_clusters=i, random_state=98)
    km.fit(df_pca)
    inertia.append(km.inertia_)
    
elbow_data = {'k': k, 'inertia': inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x='k', y='inertia', xticks = k, title='elbow curve')

C:\Users\janze\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
C:\Users\janze\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
C:\Users\janze\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(
C:\Users\janze\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Window

:Curve   [k]   (inertia)

Running K-Means with `k=4`

In [36]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=98)

# Fit the model
model.fit(df_pca)

# Predict clusters
predictions = model.predict(df_pca)
print(predictions)

[0 0 0 2 2 2 0 2 2 2 0 2 0 0 2 0 2 2 0 0 2 2 2 2 2 0 2 2 2 0 2 0 2 2 0 0 2
 2 2 2 2 2 0 0 2 2 2 2 2 0 0 2 0 2 2 2 2 0 2 2 0 2 0 0 0 2 2 2 0 0 0 0 0 2
 2 2 0 0 2 0 2 0 0 2 2 2 2 0 0 2 0 2 2 0 0 2 0 0 2 2 0 0 2 0 0 2 0 2 0 2 0
 2 0 0 2 2 0 2 2 2 0 2 2 2 2 2 0 0 2 2 2 0 2 0 2 2 0 2 0 2 0 0 2 2 0 2 2 0
 0 2 0 2 0 0 0 2 2 2 2 0 0 0 0 0 2 2 0 0 0 0 0 2 0 0 0 0 0 2 0 2 0 0 2 0 2
 0 0 2 0 2 0 2 0 2 0 0 0 0 2 0 0 0 0 0 2 2 0 0 2 2 0 0 0 0 0 2 0 0 0 0 0 0
 0 0 2 0 0 0 0 0 0 2 2 2 0 0 0 0 2 0 2 0 0 2 0 2 2 0 2 2 0 2 0 0 0 2 0 0 2
 0 0 0 0 0 0 0 2 0 2 0 0 0 0 2 0 2 0 2 2 2 2 0 2 0 0 2 0 2 2 2 0 2 0 2 2 2
 0 2 0 2 0 0 0 2 0 2 2 2 2 2 0 0 2 0 0 0 2 0 2 0 2 0 2 0 0 0 0 2 0 0 2 0 0
 0 2 2 2 2 0 0 0 0 2 0 2 2 2 0 0 2 2 0 0 2 0 2 2 2 0 2 2 0 0 0 2 2 2 0 0 0
 2 2 0 2 2 2 2 0 3 3 2 2 2 0 0 0 0 0 0 2 2 2 2 0 0 0 2 0 2 0 0 0 0 2 0 0 2
 0 0 2 2 0 2 0 2 2 2 2 0 0 2 0 2 0 0 0 0 0 0 2 2 2 0 0 0 0 0 0 2 0 2 2 2 2
 0 0 0 0 2 0 0 2 0 0 2 0 2 0 2 2 0 0 2 0 2 2 0 2 2 0 2 0 2 0 0 2 0 0 0 0 0
 2 2 2 0 0 0 2 0 2 0 2 0 

C:\Users\janze\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1332: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


In [37]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([df, pcs_df], axis=1, join='inner')

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df['CoinName'] = df_names

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df['Class'] = predictions

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

NameError: name 'crypto_df' is not defined

### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [ ]:
# Creating a 3D-Scatter with the PCA data and the clusters
import plotly.express as px
fig = px.scatter_3d(clustered_df, x='PC 1', y='PC 2', z='PC 3',
              color='Class',
              hover_name='CoinName'
              )

fig.show()


In [ ]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(["CoinName", "Algorithm", "ProofType", "MaxSupply", "TotalCoinsMined", "Class"])

In [ ]:
# Print the total number of tradable cryptocurrencies.
num_tradable_coins = clustered_df['CoinName'].count()
print(f'Tradable coins: {num_tradable_coins}')

In [ ]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
clustered_df.hvplot.scatter(
    x="PC 1",
    y="PC 2" ,
    by='Class',
    hover_cols=["CoinName"] ,
    title = "PC Clusters 1/2"
) + clustered_df.hvplot.scatter(
    x="PC 1",
    y="PC 3" ,
    by='Class',
    hover_cols=["CoinName"],
    title = "PC Clusters 1/3"
) + clustered_df.hvplot.scatter(
    x="PC 2",
    y="PC 3" ,
    by='Class',
    hover_cols=["CoinName"],
    title = "PC Clusters 2/3"
)

In [ ]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# YOUR CODE HERE

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# YOUR CODE HERE

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
# YOUR CODE HERE

plot_df.head(10)

In [ ]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
